In [ ]:
!pip install transformers
!pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


English Data

### Mixing english and tamil translated to english data

In [3]:
import pandas as pd
concatenated=pd.read_csv('/content/drive/MyDrive/NLP-Project/BalencedLGBT_dataset.csv')
concatenated['category'] = concatenated['category'].replace('Non-anti-LGBT+ content', 'ally')
concatenated['category'] = concatenated['category'].replace('ally', 1)
concatenated['category'] = concatenated['category'].replace('Homophobic', 0)
concatenated['category'] = concatenated['category'].replace('Transphobic', 0)

In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5331, 2)
Validation set shape: (1777, 2)
Test set shape: (1778, 2)


In [5]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

0    3536
1    1795
Name: category, dtype: int64
0    1179
1     598
Name: category, dtype: int64
0    1180
1     598
Name: category, dtype: int64


In [6]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [7]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234911 sha256=1f9224f87b5424cbeab41f92ee37206916da414ffb21366e89e87c8a538f98ec
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [8]:

import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [9]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5,
             "manual_seed": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [10]:
# Train the model
model.train_model(train_df)

  0%|          | 0/5331 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

(501, 0.24426408564326113)

Validation Set


In [11]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1777 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

{'mcc': 0.8289041308044076,
 'f1': 0.9144463324837158,
 'acc': 0.923466516601013,
 'eval_loss': 0.25633497354907114}

Testing With English Data

In [12]:
import pandas as pd
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()
test_df = e_test
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('ally', 1)
test_df['category'] = test_df['category'].replace('Homophobic', 0)
test_df['category'] = test_df['category'].replace('Transphobic', 0)
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [18]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak about culture., pre marita...",1,0
1,Really this prank will be more powerful Feedba...,1,0
2,Can you share your thoughts on cannabis and th...,1,0
3,In the suspension meaning West ...,1,0
4,it's purely biological !,1,0
...,...,...,...
60,Give equilism to all human being. slightly_sm...,1,0
61,But according to hindu scriptures it is not. ...,1,0
62,Mr Tomboy she is lesbian ?? ...,1,0
63,Solute for everybady.....,1,0


In [19]:
mis.predicted_label.value_counts()

0    49
1    16
Name: predicted_label, dtype: int64

In [20]:
from sklearn.metrics import classification_report
target_names = [ 'neg', 'pos']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

         neg       0.51      0.76      0.61        66
         pos       0.98      0.95      0.96       924

    accuracy                           0.93       990
   macro avg       0.74      0.85      0.79       990
weighted avg       0.95      0.93      0.94       990



##Analysing sentence wise